In [1]:
### imports

# external modules
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import DataLoader
importlib.reload(DataLoader)
import HistStruct
importlib.reload(HistStruct)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)
import HistStruct
importlib.reload(HistStruct)
import BuildAutoencoder
importlib.reload(BuildAutoencoder)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)

2022-06-09 18:19:18.775568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.

<module 'AutoEncoder' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/classifiers/AutoEncoder.py'>

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = False

# Control for the notebook - turn off user-friendly mode to enable faster runtimes
userfriendly = False

In [3]:
### Data Controls and Selection

# The directory data is located in
datadir = '../data/'

# Select a list of good runs to train on
# Should be validated by eye
runs = [ 
    297057, 297099, 297101, 297113, 
    297114, 297175, 297177, 297179, 
    297215, 297218, 297225, 297296, 
    297411, 297426, 297431, 297434,
    297468, 297483, 297486, 297503,
    297557, 297598, 297604, 297620, 
    297659, 297670, 297674, 297678, 
    297722, 298997, 299061, 299065, 
    299067, 299096, 299149, 299178, 
    299184, 299185, 299327, 299329, 
    299480]


# The year and eras being used
year = '2017'
eras = ['B']

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [
    ['NormalizedHitResiduals_TIB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2',
     'NormalizedHitResiduals_TIB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3' , 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'],
    ['chargeInner_PXLayer_1','chargeOuter_PXLayer_1'],
    ['chargeInner_PXLayer_2', 'chargeOuter_PXLayer_2']
]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [4]:
### Model Controls and Selection

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

# imports
from keras.models import load_model

# Evaluate models seperately, as an ensemble, both, or neither
individualEval = True
ensembleEval = False

In [5]:
### Define a Test Set
goodrunsls = { 
                #"297056":[[-1]],
                #"297177":[[-1]],
                #"301449":[[-1]]  
                "297050":[[-1]],
                "297474":[[-1]],
                "299000":[[-1]],
                "297057":[[-1]],
                "297177":[[-1]],
                "297296":[[-1]]
             }
badrunsls = {
                #"297048":[[-1]],
                #"297282":[[-1]],
                #"297283":[[-1]],
                #"297284":[[-1]],
                #"297287":[[-1]],
                #"297288":[[-1]],
                #"297289":[[-1]],
                #"299316":[[-1]],
                #"299317":[[-1]],
                #"299318":[[-1]],
                #"299324":[[-1]],
                #"299326":[[-1]],
                #"301086":[[88,126]],
                #"301086":[[89,89]],
                #"303948":[[1710,1710]],
                "297047":[[-1]], #close but, true bad for all 8
                #"297169":[[-1]], #true bad for all 8
               #"297211":[[-1]], #Reconstructs well
               #"299325":[[-1]], #Reconstructs well
               #"297664":[[-1]],#true bad for all 8
              # "299317":[[-1]], #true bad for all 8
               #"297169":[[-1]]#true bad for all 8
            }


In [6]:
### Function to prep data for combined autoencoder
# This function may need alteration if using a concatenate layer
def prep_combine_hists(histnames):
    
    if readnew:
        
        # Initializations
        dloader = DataLoader.DataLoader()
        dataframes = []
        
        # Loop over eras
        for era in eras:
            # Loop over histogram types to initiate merge where needed
            if userfriendly:
                print('Processing data for era ' + str(era))
            for histnamels in histnames:
                if userfriendly:
                    print('Adding {}...'.format(histnamels))
                # Combine any desired histograms into 1D
                df = prep_1D_df(histnamels, era)
                # Store histograms
                dataframes.append(df)
        if userfriendly:
            print('Autoencoders to train: ' + str(len(dataframes)))
    # Currently unavailble functionality 
#    if save:
#        histstruct.save( 'test.pkl' )
#        
#    if not readnew:
#        histstruct = HistStruct.HistStruct.load( 'test.pkl' )
#        nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
#
#    print('Created a histstruct with the following properties:')
#    print('- number of histogram types: {}'.format(len(histstruct.histnames)))
#    print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))
#    print('- masks: {}'.format(list(histstruct.masks.keys())))
    
    return dataframes

In [7]:
### Function to prep dataframe for 1D autoencoder
# This function may soon be obsoleted by using a concatenate layer in the neural network
def prep_1D_df(histnamels, era):
    
    # Creates a list of csv filenames for the histograms
    csv_names = []
    for histname in histnamels:
        filename = datadir + 'DF' + year + era + '_' + histname + '.csv'
        csv_names.append(filename)
    
    # Combines the CSV files into a single dataframe
    df1 = csvu.read_and_merge_csv(csv_names, userfriendly)
    
    # Stitch the histograms together into a single 1D histogram
    df1 = dfu.merge_ls(df1)
    df1.reset_index(drop=True,inplace=True)
    
    # Print information about the data
    if(userfriendly): print('\nRaw input data shape: {}'.format( dfu.get_hist_values(df1)[0].shape))
    
    # Select out reference runs from dataset
    df2 = dfu.select_runs(df1, runs)
    
    # Filter the dataset to have DCS-bit on and high statistics
    df2 = dfu.select_dcson(df2)
    if userfriendly: print('Number of passing lumisections after DCS selection: {}'.format( len(df2) ))
    
    df2 = dfu.select_highstat(df2,entries_to_bins_ratio=100)
    if userfriendly: print('Number of passing lumisections after high statistics selection: {}\n'.format( len(df2)))
        
    return df1, df2

In [8]:
### Prepare Testing Data
def prep_input_data(dataframes):
    rebinningfactor = 1

    # Convert dataframes into histograms
    X_trainls = []
    iterate = 0
    for df in dataframes:
        if userfriendly: print('Preparing data from dataframe ' + str(iterate))
        X_trainls.append(hu.preparedatafromdf(df[1],rebinningfactor=rebinningfactor,donormalize=True,doplot=createPlots))
        if userfriendly: print('Size of training set for autoencoder {}: '.format(iterate)
             + str(X_trainls[iterate].shape))
        plt.show()
        iterate = iterate + 1
    
    return X_trainls

In [9]:
### Model Training/Loading
def train_mashed_autoencoder(X_trainls):
    models = []
    lossplots = []
    iterate = 0
    for X_train in X_trainls:
        iterate = iterate + 1
        
        # Train a new model
        if trainnew:
            if userfriendly: print('Now training model ' + str(iterate) + "/" +
                 str(len(X_trainls)))
            
            # Model parameters
            input_size = X_train.shape[1]
            arch = [int(X_train.shape[1]/2.)]
            act = ['tanh']*len(arch)
            opt = 'adam'
            loss = aeu.mseTop10
            if userfriendly: 
                verbose = 1 
                print(str(X_train.shape))
            else: verbose = 0
            callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
            
            # Model creation and training
            autoencoder = aeu.getautoencoder(input_size,arch,act,opt,loss)
            history = autoencoder.fit(X_train, X_train, epochs=200, batch_size=20, shuffle=True, verbose=verbose, validation_split=0.1, callbacks=[callback])
            
            # Save the model and loss function in memory for reference
            models.append(autoencoder)
            lossplots.append(history)
        
            # Case of saving the model
            if savemodel: autoencoder.save(modelname)
             
    # Return trained model and lossplots for evaluation
    return [models, lossplots]
        
        # Currently unsupported function    
        # Load existing model
    #    else:
    #        # autoencoder = load_model(modelname+'/saved_model.pb',custom_objects={'mseTop10': aeu.mseTop10})
    #        autoencoder = tf.keras.models.load_model(modelname ,custom_objects={'mseTop10': aeu.mseTop10})

In [27]:
### Evaluation of Models on Training Set
def evaluate_autoencoders_training(X_trainls, models, lossplots):
    predictionTrainls = []
    mseTrainls = []
    meanls = []
    stdls = []
    
    for i in range(len(models)):
        if trainnew and createPlots:
            # Plot model loss function
            plot = pu.plot_loss(lossplots[i], title = 'model loss')
            plt.savefig(name+'_loss.png')
            plt.show()
            
        # Get data for evaluation
        predictionTrain = models[i].predict(X_trainls[i])
        mseTrain = aeu.mseTop10Raw(X_trainls[i], predictionTrain)
        print(predictionTrain.shape)
        
        # Store evaluation data for later access
        predictionTrainls.append(predictionTrain)
        mseTrainls.append(mseTrain)
        
        # Plot Global MSE Trends
        if createPlots:
            plot = pu.plot_mse(mseTrain, rmlargest = 0.01)
            plot[1].legend()
            plt.savefig(name+'_mse.png', bbox_inches="tight")
            plt.show()
        
        # Get mean and standard deviation
        (mean,std) = pu.plot_mse(mseTrain,doplot=False,rmlargest=0.01)
        meanls.append(mean)
        stdls.append(std)
        if userfriendly:
            print('mean mse: {}'.format(mean))
            print('std mse: {}'.format(std))
        
    return predictionTrainls, meanls, stdls, mseTrainls

In [11]:
### Define Working Point for Train Data
def workingpoint_on_train_data(models, X_trainls, predictionTrainls,
                              meanls, stdls, mseTrainls):
    for i in range(len(models)):
        
        # Making code compatible with code from other notebooks
        X_train = X_trainls[i]
        predictionTrain = predictionTrainls[i]
        mean = meanls[i]
        std = stdls[i]
        mseTrain = mseTrainls[i]
    
        cutvalue = mean + 3*std
        #cutvalue = 4.73e-6
        if userfriendly: print('The mse threshold is: '+str(cutvalue))
        goodindices = np.arange(0,len(mseTrain))[mseTrain<cutvalue]
        badindices = np.arange(0,len(mseTrain))[mseTrain>cutvalue]
        
        #test
        #print(goodindices)
        #print(badindices)
        #end test
        
        if userfriendly:
            print('Number of passing histograms: '+str(len(goodindices)))
            print('Number of failing histograms: '+str(len(badindices)))
        
        if createPlots:
            nplot = 3
            
            if len(goodindices) >= nplot:
                print('Examples of good histograms and reconstruction:')
                randint = np.random.choice(goodindices,size=nplot,replace=False)
                
                for j in randint: 
                    histlist = [X_train[int(j),:],predictionTrain[int(j),:]]
                    labellist = ['data','reconstruction']
                    colorlist = ['black','blue']
                    pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                    plt.show()
             
            # Warn the user to try a better cutvalue
            else:
                print('\nNot enough good examples, choose a less sensitive cutvalue.\n')
                
            if len(badindices) >= nplot:
                print('Examples of bad histograms and reconstruction:')
                randint = np.random.choice(badindices,size=nplot,replace=False)
                for j in randint:
                    histlist = [X_train[int(j),:],predictionTrain[int(j),:]]
                    labellist = ['data','reconstruction']
                    colorlist = ['black','blue']
                    pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                    plt.show()
                
            # Warn the user to try a better cutvalue
            else:
                print('\nNot enough bad examples, choose a more sensitive cutvalue\n')

In [12]:
### Preparing Testing Data
def prep_wp_data(dataframes):
    X_testls = []
    for df in dataframes:
        df1 = dfu.select_runs(df[0], [
                297050,  
                297474, 
                299000, 
                299317,
                297047,
                297169,
                297211,
                299325,
                297664, 
                297057,
                297177,
                297296])
        
        # Same preprocessing of DCS-bit on and High Statistics
        df1 = dfu.select_dcson(df1)
        df1 = dfu.select_highstat(df1,entries_to_bins_ratio=100)
        
        # Prep histograms
        X_test_good = hu.preparedatafromdf( dfu.select_runsls(df1,goodrunsls),donormalize=True )
        X_test_bad = hu.preparedatafromdf( dfu.select_runsls(df1,badrunsls),donormalize=True )
        
        # Plotting the good and bad data
        if createPlots:
            pu.plot_sets([X_test_good,X_test_bad],colorlist=['b','r'],
            labellist=['Histograms in test set labeled "good"','Histograms in test set labeled "bad"'])
            
        X_testls.append([X_test_good, X_test_bad])
    
    # Returns a list of good and bad data from all the dataframes
    return X_testls

In [25]:
### Creates predictions from the autoencoders and evaulates their performance
def predict_autoencoders_train(X_testls, models):
        
    mse_good = []
    mse_bad = []
    for i in range(len(models)):
        if userfriendly: print('\nEvaluating model ' + str(i + 1) + '/' + str(len(models)))
        
        # Defining variables for modularity with other people's code
        X_test_good = X_testls[i][0]
        X_test_bad = X_testls[i][1]
        autoencoder = models[i]
        
        # Getting evaluation criteria
        prediction_test_good = autoencoder.predict(X_test_good)
        mse_test_good = aeu.mseTopNRaw(X_test_good, prediction_test_good, n=10 )
        prediction_test_bad = autoencoder.predict(X_test_bad)
        mse_test_bad = aeu.mseTopNRaw(X_test_bad, prediction_test_bad, n=10 )
        
        # Taking criteria to find a working point later
        mse_good.append(mse_test_good)
        mse_bad.append(mse_test_bad)
        
        if userfriendly:
            print('Average MSE on good set: ' + str(np.mean(mse_test_good)))
            print('Average MSE on bad set: ' + str(np.mean(mse_test_bad)))
        
        if createPlots:
            # Number of plots of each type to generate per model (so nplot * 2 * len(model))
            nplot = 3
            
            # Good examples
            print('Examples of good histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_good)),size=nplot,replace=False)
            for i in randint: 
                histlist = [X_test_good[int(i),:],prediction_test_good[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
            
            # Bad examples
            print('Examples of bad histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_bad)),size=nplot,replace=False)
            for i in randint:
                histlist = [X_test_bad[int(i),:],prediction_test_bad[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
               
        mse_good_eval = np.array(mse_good)
        mse_bad_eval = np.array(mse_bad)
        evaluate_ensemble_test(mse_good_eval, mse_bad_eval, models[i])

In [14]:
### Evaluate the models together
def evaluate_ensemble_test(mse_good, mse_bad, autoencoder):
    

In [15]:
### Get some metrics from the data
def get_metrics(hists, labels, predicted_hists, msewp=None):
    scores = aeu.mseTop10Raw(hists, predicted_hists)
    wp = msewp
    
    nsig = np.sum(labels)
    nback = np.sum(1-labels)
    
    tp = np.sum(np.where((labels==1) & (scores>wp),1,0))/nsig
    fp = np.sum(np.where((labels==0) & (scores>wp),1,0))/nback
    tn = 1-fp
    fn = 1-tp
    
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*precision*recall) / (precision + recall)
    return [accuracy, precision, recall, f1]

In [16]:
### Preprocessing Data

# Setting up data for mixed type autoencoder
dataframes = prep_combine_hists(histnames)

INFO in csv_utils.py / read_and_merge_csv: merged 8 csv files.
Assigning name: NormalizedHitResiduals_TIB__Layer__1
INFO in csv_utils.py / read_and_merge_csv: merged 2 csv files.
Assigning name: chargeInner_PXLayer_1
INFO in csv_utils.py / read_and_merge_csv: merged 2 csv files.
Assigning name: chargeInner_PXLayer_2


In [17]:
### Preparing Data for Input
X_trainls = prep_input_data(dataframes)

In [18]:
### Building the Model
(models, lossplots) = train_mashed_autoencoder(X_trainls)

2022-06-09 18:19:51.311951: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.cern.ch/lc

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 408)               333336    
_________________________________________________________________
dense_1 (Dense)              (None, 816)               333744    
Total params: 667,080
Trainable params: 667,080
Non-trainable params: 0
_________________________________________________________________


2022-06-09 18:19:51.686637: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-09 18:19:51.688578: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2900000000 Hz


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When f

2022-06-09 18:20:18.725561: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Test/assets
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 102)               20910     
_________________________________________________________________
dense_3 (Dense)              (None, 204)               21012     
Total params: 41,922
Trainable params: 41,922
Non-trainable params: 0
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the f

In [28]:
### Autoencoder Evaluation
(predictionTrainls, meanls, stdls, mseTrainls, fitfunc) = evaluate_autoencoders_training(X_trainls, models, lossplots)

(10563, 816)
(10553, 204)
(10539, 204)


/tmp/ipykernel_4125/2359259832.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mse_fit = np.array(mseTrainls)


Exception: ERROR in CloudFitter.init: points must be a 2D numpy array but found shape (3,)

In [20]:
### Working Point Definition
workingpoint_on_train_data(models, X_trainls, predictionTrainls,
                              meanls, stdls, mseTrainls)

In [22]:
### Preparing Test Data
X_testls = prep_wp_data(dataframes)

In [26]:
predict_autoencoders_train(X_testls, models)

Done
Done
Done


/tmp/ipykernel_4125/3634279960.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mse_good_eval = np.array(mse_good)
